# Setup CW

In [220]:
import chipwhisperer as cw
scope = cw.scope()
target = cw.target(scope)
scope.default_setup()

# Setup Scope

In [341]:
from tqdm.notebook import trange
import numpy as np

scope.io.tio1 = 'serial_tx'
scope.io.tio2 = 'serial_rx'
scope.trigger.triggers = "nrst"

scope.gain.db = 56
scope.adc.samples = 11500
scope.adc.offset = 0
scope.adc.decimate = 1
scope.clock.clkgen_freq = 4000000 
N = 100000

cmd=bytearray([255]*1)

ktp = cw.ktp.Basic() 
ktp.setInitialKey("54 ee ff 11 21 32 eb 43 bb aa ff 23 98 54 12 12")

# Collect Traces

In [343]:
#%run "../swaes/real_time_plot.ipynb"
#plot = real_time_plot(plot_len=scope.adc.samples-1)

In [338]:
traces_enc = []
text_arrays = []
key_arrays = []

In [346]:
for i in trange (N-len(traces_enc)):
    scope.arm()
    key, text = ktp.next() 
    target.write(cmd+text)
    ret = scope.capture()
    if ret:
        print("Target timed out!")
        break
    trace = scope.get_last_trace()
    #plot.send(trace)
    traces_enc.append(np.asarray(trace))
    text_arrays.append(np.asarray(text))
    key_arrays.append(np.asarray(key))

# Save as bin

In [349]:
np.array(traces_enc).tofile("traces_enc.bin")

In [350]:
np.array(text_arrays).tofile("text_arrays.bin")

In [351]:
np.array(key_arrays).tofile("key_arrays.bin")

In [345]:
#%matplotlib inline
#import matplotlib.pyplot as plt
#plt.figure()
#plt.plot(traces_enc[0])
#plt.show()